In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, ln, hour,abs,col

from pyspark.ml.feature import Tokenizer, RegexTokenizer, NGram, HashingTF, ChiSqSelector, VectorAssembler

from pyspark.ml.classification import LogisticRegression

In [4]:
spark = SparkSession.builder.appName("NYC").master("local[4]").getOrCreate()
sc = spark.sparkContext

25/02/05 16:09:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
train = spark.read.csv("train80-rich.csv",header=True,inferSchema=True).repartition(8)

train.show(3)

train.registerTempTable("train")

+---------+---------+-------------------+-------------------+---------------+------------------+-----------------+------------------+-----------------+------------------+-------------+------------------+----+-----+------------------+------------------+
|       id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|  pickup_longitude|  pickup_latitude| dropoff_longitude| dropoff_latitude|store_and_fwd_flag|trip_duration|             ln_td|hh24|month|                l2|                l1|
+---------+---------+-------------------+-------------------+---------------+------------------+-----------------+------------------+-----------------+------------------+-------------+------------------+----+-----+------------------+------------------+
|id0116119|        2|2016-01-28 17:13:44|2016-01-28 17:16:19|              5|-73.98477935791016|40.74222183227539|-73.98902893066406|40.74160385131836|                 N|          155|2.1903316981702914|  17|    1|365.43124774828914|  541.68

/Users/husnusensoy/Documents/code/tt-bootcamp/.venv/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [6]:
spark.sql("select mean(trip_duration),count(1) as med_duration from train").show()

[Stage 5:>                                                          (0 + 2) / 2]

+-------------------+------------+
|mean(trip_duration)|med_duration|
+-------------------+------------+
|   960.065212976095|     1166915|
+-------------------+------------+



In [7]:
test = spark.read.csv("test20-rich.csv",header=True,inferSchema=True).repartition(8)

test.show(3)

test.registerTempTable("test")

[Stage 13:=============================>                            (1 + 1) / 2]

+---------+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+----+-----+------------------+------------------+
|       id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|  pickup_longitude|   pickup_latitude| dropoff_longitude|  dropoff_latitude|store_and_fwd_flag|trip_duration|hh24|month|                l2|                l1|
+---------+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+----+-----+------------------+------------------+
|id0030599|        2|2016-05-10 13:20:00|2016-05-10 13:22:13|              5|-73.95597076416016|  40.7791862487793|-73.95756530761719| 40.77682113647461|                 N|          133|  13|    5| 295.1290642467485| 440.1485867942956|
|id0745079|        2|2016-05-02 07:07:34|2016-05-02 07:2

In [6]:
spark.sql("select avg(power(trip_duration - 960,2)) as error from test").show()

+-------------------+
|              error|
+-------------------+
|1.030098445489977E7|
+-------------------+



### Some Categorical Things

In [7]:
spark.sql("""
select vendor_id, hour(pickup_datetime) hh, mean(trip_duration) avg_dur, count(1) n 
from train 
group by 1,2
""").registerTempTable("summary_stats")

In [8]:
spark.sql("""
select a.trip_duration,b.avg_dur 
from test a 
join summary_stats b on (a.vendor_id = b.vendor_id and hour(a.pickup_datetime) = b.hh) 
order by abs(a.trip_duration - avg_dur) desc
""").show()

[Stage 27:=============================>                            (4 + 2) / 8]

+-------------+------------------+
|trip_duration|           avg_dur|
+-------------+------------------+
|        86346| 766.5796019900497|
|        86286| 910.2591969020962|
|        86365| 989.7060029665823|
|        86357| 981.7420898329461|
|        86353| 981.7420898329461|
|        86348|  983.554221801336|
|        86392|1029.2058158856762|
|        86332| 984.1358433416688|
|        86332| 984.1358433416688|
|        86331| 984.1358433416688|
|        86334| 989.6070192782995|
|        86367|1024.2281807901861|
|        86323|  983.554221801336|
|        86317| 984.1358433416688|
|        86312|  983.554221801336|
|        86346|1024.2281807901861|
|        86303| 981.7420898329461|
|        86301|  983.554221801336|
|        86307| 989.7060029665823|
|        86304| 989.6070192782995|
+-------------+------------------+
only showing top 20 rows



In [9]:
spark.sql("""
select avg(pow(a.trip_duration - b.avg_dur ,2))
from test a 
join summary_stats b on (a.vendor_id = b.vendor_id and hour(a.pickup_datetime) = b.hh) 
""").show()

[Stage 36:==================================================>       (7 + 1) / 8]

+--------------------------------------+
|avg(pow((trip_duration - avg_dur), 2))|
+--------------------------------------+
|                  1.0282322632651418E7|
+--------------------------------------+



In [19]:
from pyspark.ml.regression import LinearRegression,GBTRegressor
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.clustering import KMeans

In [43]:
pickup_cluster = Pipeline(stages=[VectorAssembler(inputCols=["pickup_longitude","pickup_latitude"]
                                                  , outputCol="pick_point"),
                     KMeans(featuresCol="pick_point", predictionCol="pickup_cluster",k=20)])

dropoff_cluster = Pipeline(stages=[VectorAssembler(inputCols=["dropoff_longitude","dropoff_latitude"]
                                                  , outputCol="dropoff_point"),
                     KMeans(featuresCol="dropoff_point", predictionCol="dropoff_cluster",k=20)])

In [56]:
ppl = Pipeline(stages=[pickup_cluster,
                       dropoff_cluster, 
                       VectorAssembler(inputCols=["vendor_id", "passenger_count", "l1","l2","hh24", "pickup_cluster","dropoff_cluster"]
                                       , outputCol="features"),
                       GBTRegressor(labelCol="trip_duration", maxDepth=2)])

In [57]:
ppl_model = ppl.fit(train)

In [58]:
ppl_model.transform(test).select("trip_duration", "prediction").registerTempTable("first_ol")

In [59]:
spark.sql("select trip_duration, prediction from  first_ol order by abs(trip_duration-prediction) desc ").show(10)

[Stage 3857:==========================================>             (6 + 2) / 8]

+-------------+-----------------+
|trip_duration|       prediction|
+-------------+-----------------+
|        86334|487.7756687941402|
|        86323|533.4635961649492|
|        86292|503.5695650425463|
|        86286|507.4833716638013|
|        86367|596.9373596553418|
|        86332|567.9908696494623|
|        86262|507.4833716638013|
|        86362|616.7542452418478|
|        86321|583.7972295772214|
|        86356| 619.767021971617|
+-------------+-----------------+
only showing top 10 rows



In [60]:
spark.sql("select sqrt(avg(power(trip_duration-prediction,2))) from  first_ol ").show()

[Stage 3860:==========================================>             (6 + 2) / 8]

+-------------------------------------------------+
|SQRT(avg(POWER((trip_duration - prediction), 2)))|
+-------------------------------------------------+
|                                3162.363645882543|
+-------------------------------------------------+

